In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
%cd / content/gdrive/MyDrive/cs394n_project/CS394N
! pip3 install -r requirements.txt

In [ ]:
# Update path for custom module support in Google Colab
import sys
sys.path.append('/content/gdrive/MyDrive/cs394n_project/CS394N/src')

In [ ]:
import torch
import matplotlib.pyplot as plt
import numpy as np

from torch.utils.data import DataLoader

from torchvision import transforms
from torchvision.datasets import CIFAR10, FashionMNIST

from tqdm.autonotebook import tqdm, trange

from utils.nets import *
from utils.model_tools import train, test, get_recall_per_epoch
from utils.dataset_tools import split_training_data, reorder_classes

In [ ]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

# Data Preparation

In [ ]:
model_selection = 'linear' # linear | cnn | cnn-demo | vgg
dataset_selection = 'fashionmnist' # cifar10 | fashionmnist

In [ ]:
if dataset_selection == 'fashionmnist':
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5), (0.5))]) # Images are grayscale -> 1 channel
else:
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

## Load Dataset

In [ ]:
train_data = FashionMNIST(root='./data', train=True, download=True, transform=transform)
test_data = FashionMNIST(root='./data', train=False, download=True, transform=transform)
    
total_classes = len(np.unique(train_data.targets))

## Reorder Classes

In [ ]:
# FashionMNIST (torchvision & paper)
# ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat", "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]

In [ ]:
help(reorder_classes)

In [ ]:
targets, classes = reorder_classes(train_data, ordering)
train_data.targets = targets
train_data.classes = classes

targets, classes = reorder_classes(test_data, ordering)
test_data.targets = targets
test_data.classes = classes

## Create Subsets

In [ ]:
holdout_classes = [8, 9]
batch_size = 32

In [ ]:
included_data, excluded_data = split_training_data(train_data, holdout_classes) 

train_inc_loader = DataLoader(included_data, batch_size=batch_size, shuffle=True, num_workers=2)
train_exc_loader = DataLoader(included_data, batch_size=batch_size, shuffle=True, num_workers=2)

In [ ]:
included_data, excluded_data = split_training_data(test_data, holdout_classes)

test_inc_loader = DataLoader(included_data, batch_size=batch_size, shuffle=True, num_workers=2)
test_exc_loader = DataLoader(included_data, batch_size=batch_size, shuffle=True, num_workers=2)

# Train Model

## Load Architecture

In [ ]:
num_classes = total_classes - len(holdout_classes)

if model_selection == 'linear':
    input_size = train_data[0][0].shape[0] * train_data[0][0].shape[1] * train_data[0][0].shape[2]
    model = LinearFashionMNIST_alt(input_size, num_classes)
elif model_selection == 'cnn':
    model = CNN_6L(num_classes)
elif model_selction == 'cnn-demo':
    model = CNN_demo(num_classes)
elif model_slection == 'vgg':
    print('Model not implemented')
    
model.to(device)

## File Paths

In [ ]:
weight_dir = './models/'
log_dir = './logs/'

model_file = weight_dir + model_selection + '_' + dataset_selection + '_' + 'holdout' + '_' + str(holdout_classes) + '.pt'
recall_file = log_dir + model_selection + '_' + dataset_selection + '_' + 'holdout' + '_' + str(holdout_classes) + 'recall.npy'
train_losses_file = log_dir + model_selection + '_' + dataset_selection + '_' + 'holdout' + '_' + str(holdout_classes) + 'train_loss.txt'
test_losses_file = log_dir + model_selection + '_' + dataset_selection + '_' + 'holdout' + '_' + str(holdout_classes) + 'test_loss.txt'

## Hyperparameters

In [ ]:
num_epochs = 15

initial_learning_rate = 0.001
final_learning_rate = 0.0001

# initial_lr * decay_rate^num_epochs = final_lr
decay_rate = (final_learning_rate/initial_learning_rate)**(1/num_epochs)

loss_fn = torch.nn.CrossEntropyLoss()
#optimizer = torch.optim.SGD(model.parameters(), lr=initial_learning_rate)
optimizer = torch.optim.Adam(model.parameters(), lr=initial_learning_rate)
#optimizer = torch.optim.AdamW(model.parameters(), lr=initial_learning_rate)
lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer=optimizer, gamma=decay_rate)

## Training Loop

In [ ]:
train_losses = []
test_losses = []
#t = trange(num_epochs)
t = range(num_epochs)
y_preds = []
y_actuals = []

for epoch in t:
    print(f"Epoch {epoch+1}\n-------------------------------")
    train_loss = train(train_inc_loader, model, loss_fn, optimizer, device)
    test_loss, y_pred, y_actual = test(test_inc_loader, model, loss_fn, device)
    print(y_pred[:2])
    train_losses.append(train_loss)
    test_losses.append(test_loss)
    y_preds.append(y_pred)
    y_actuals.append(y_actual)
    
    #t.set_description(f"Epoch {epoch} train loss: {epoch_loss_train[-1]:.3f}")
    lr_scheduler.step()
    
torch.save(model.state_dict(), model_file)

recalls = get_recall_per_epoch(y_actuals, y_preds, num_classes)
np.save(recall_file, recalls)

with open(train_losses_file, 'w') as fp:
    for s in train_losses:
        fp.write("%s\n" % s)
        
with open(test_losses_file, 'w') as fp:
    for x in test_losses:
        fp.write("%s\n" % x)

print("Done!")

In [ ]:
recalls_loaded = np.load(recall_file)
print(recalls == recalls_loaded)
# plots